#Descrição do Projeto
O projeto trata-se de um sistema para garantir a segurança dos operários de máquinas pesadas, principalmente sobre o uso adequado de EPI. Utiliza tecnologias como:

*   Python;
*   Arduino;
*   Dashboard (HTML, CSS E JS);
*   Geração e leitura de QR Code;
*   Serialização e desserialização de dados (JSON);
*   Integração de API com Flask.

\
O sistema recebe as imagens de uma câmera, lê um QR Code que contém as informações do funcionário (nome, curso e validade), valida os dados e, se estiver tudo certo, libera o acesso. Caso haja problema (curso vencido), envia alerta ao supervisor e aciona a sirene. Tudo está integrado a um dashboard que atualiza as imagens e o diagnóstico final em tempo real.

##Instalação de bibliotecas

Bibliotecas python para o desenvolvimento do sistema. Bibliotecas utilizadas:

*   Flask (API);
*   OpenCV (Captura de imagem);
*   Pybzar e pillow (Leitura de QR Code);
*   Qrcode[pil] (Geração de QR Code);
*   Pyserial (Comunicação com Arduino).

In [1]:
pip install flask opencv-python pyzbar pillow qrcode[pil] pyserial

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00


##Geração de QR Codes

Criação dos QR Codes que vão estar nos crachás dos funcionários para validação dos seus respectivos dados:

*   Nome;
*   Curso para operar o maquinário;
*   Data de vencimento do curso.


In [36]:
import qrcode, json

funcionario = {
    "nome": "Maria Eduarda",
    "curso": "sim",
    "validade": "2025-10-20"
}

qr = qrcode.make(json.dumps(funcionario))
qr.save("qr_eduarda.png")
print("QR Code gerado!")

QR Code gerado!


##Leitura de QR Code e validação de dados


In [32]:
import cv2 as opencv
import json
from datetime import datetime

img = opencv.imread('qr_eduarda.png')
detector = opencv.QRCodeDetector()
dados, bbox, _ = detector.detectAndDecode(img)
if bbox is not None:
    print("Dados do QR Code:", dados)
else:
    print("Nenhum QR Code detectado.")

try:
    funcionario = json.loads(dados)

    nome = funcionario.get("nome", "")
    curso = funcionario.get("curso", "").strip().lower()
    validade = funcionario.get("validade", "")

    if curso != "sim":
        print(f"{nome} NÃO está autorizado! Motivo: curso não está válido.")
    else:
        validade_dt = datetime.strptime(validade, "%Y-%m-%d").date()
        hoje = datetime.now().date()

        if validade_dt >= hoje:
            print(f"{nome} está autorizado! Curso válido até {validade}.")
        else:
            print(f"{nome} está com curso vencido! (Venceu em {validade}).")

except json.JSONDecodeError:
    print("Erro: O conteúdo do QR Code não está em formato JSON válido.")
except ValueError:
    print("Erro: Data de validade está em formato inválido (esperado: YYYY-MM-DD).")

    print(f"Erro: Campo ausente no QR Code ({e}).")

Dados do QR Code: {"nome": "Daniel Nunes Medeiros", "curso": "sim", "validade": "2025-09-11"}
Daniel Nunes Medeiros está com curso vencido! (Venceu em 2025-09-11).
